In [ ]:
%pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from transformers import EvalPrediction

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/red_data.csv')
df

,Unnamed: 0,body,post_title,post_time
0,0,"Insulin, insulin, insulin, insulin.",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
1,1,The brand name drug my doctor prescribed me is...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
2,2,The fact that insulin isn't practically free i...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
3,3,You don’t need to ‘produce’ the drugs. You jus...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
4,4,"Call me a pessimist, but I'll believe it when ...",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
...,...,...,...,...
21228,21228,Now try to “change it”,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21229,21229,Why would I try to pass a law prohibiting murd...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21230,21230,Jesus Christ you are dense. Yes a law prohibit...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21231,21231,"Damn right I'm being dense, because you are to...",Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27


In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)
df

,body,post_title,post_time
0,"Insulin, insulin, insulin, insulin.",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
1,The brand name drug my doctor prescribed me is...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
2,The fact that insulin isn't practically free i...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
3,You don’t need to ‘produce’ the drugs. You jus...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
4,"Call me a pessimist, but I'll believe it when ...",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
...,...,...,...
21228,Now try to “change it”,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21229,Why would I try to pass a law prohibiting murd...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21230,Jesus Christ you are dense. Yes a law prohibit...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21231,"Damn right I'm being dense, because you are to...",Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27


In [ ]:
df2 = pd.DataFrame(df)

In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

df2['TextBlob'] = df2['body'].apply(get_sentiment)

print(df2[['TextBlob']])

       TextBlob
0       Neutral
1      Negative
2       Neutral
3      Positive
4       Neutral
...         ...
21228   Neutral
21229  Negative
21230  Positive
21231  Negative
21232   Neutral

[21233 rows x 1 columns]


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    compound = sentiment_scores['compound']

    if compound > 0:
        return 'Positive'
    elif compound < 0:
        return 'Negative'
    else:
        return 'Neutral'

df2['Vader'] = df2['body'].apply(get_vader_sentiment)

print(df2[['Vader']])

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


          Vader
0       Neutral
1      Positive
2      Negative
3      Positive
4      Negative
...         ...
21228   Neutral
21229  Negative
21230  Negative
21231  Negative
21232  Positive

[21233 rows x 1 columns]


In [ ]:
df2[['TextBlob','Vader']]

,TextBlob,Vader
0,Neutral,Neutral
1,Negative,Positive
2,Neutral,Negative
3,Positive,Positive
4,Neutral,Negative
...,...,...
21228,Neutral,Neutral
21229,Negative,Negative
21230,Positive,Negative
21231,Negative,Negative


In [ ]:
df2

,body,post_title,post_time,TextBlob,Vader
0,"Insulin, insulin, insulin, insulin.",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Neutral,Neutral
1,The brand name drug my doctor prescribed me is...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Negative,Positive
2,The fact that insulin isn't practically free i...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Neutral,Negative
3,You don’t need to ‘produce’ the drugs. You jus...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Positive,Positive
4,"Call me a pessimist, but I'll believe it when ...",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57,Neutral,Negative
...,...,...,...,...,...
21228,Now try to “change it”,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Neutral,Neutral
21229,Why would I try to pass a law prohibiting murd...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Negative,Negative
21230,Jesus Christ you are dense. Yes a law prohibit...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Positive,Negative
21231,"Damn right I'm being dense, because you are to...",Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27,Negative,Negative


# **MODEL v1.0**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/output.csv')

In [ ]:
df.columns

Index(['Unnamed: 0', 'body', 'post_title', 'post_time', 'score', 'label'], dtype='object')

In [ ]:
df = df.drop(['Unnamed: 0', 'label'], axis=1)

In [ ]:
dataset = Dataset.from_pandas(df)
train_dataset = dataset.train_test_split(test_size=0.2)['train']
val_dataset = dataset.train_test_split(test_size=0.2)['test']


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    tokenized['label'] = examples['score']
    return tokenized

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/957 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns(['text', 'score'])
val_dataset = val_dataset.remove_columns(['text', 'score'])


In [ ]:
# Set the dataset format to PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [ ]:
print(train_dataset[0])


{'input_ids': tensor([  101, 28335, 10410, 14680,   187, 10376, 10129, 40762, 62137, 25085,
        10148, 16143,   119,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
# Set up training arguments with logging and evaluation at every epoch
training_args = TrainingArguments(
    output_dir='./results',                # Output directory for saving model checkpoints
    num_train_epochs=3,                    # Number of training epochs
    per_device_train_batch_size=8,         # Batch size for training
    per_device_eval_batch_size=16,         # Batch size for evaluation
    evaluation_strategy='epoch',           # Evaluate the model at the end of each epoch
    logging_dir='./logs',                  # Directory to store logs
    logging_strategy='epoch',              # Log status after every epoch
    save_strategy='epoch',                 # Save model after every epoch
    load_best_model_at_end=True,           # Load the best model when finished
    metric_for_best_model='accuracy',      # Use accuracy as the evaluation metric
    logging_steps=10,                      # Logging steps (adjust if you want logging during training as well)
)


In [ ]:
def compute_metrics(pred: EvalPrediction):
    # Extract predictions and labels
    predictions, labels = pred.predictions, pred.label_ids
    # Get the predicted classes (assuming the output is logits)
    predictions = np.argmax(predictions, axis=1)
    # Calculate accuracy
    acc = accuracy_score(labels, predictions)
    # Calculate F1 score (you can adjust average method based on your task)
    f1 = f1_score(labels, predictions, average='weighted')
    return {
        'accuracy': acc,
        'f1_score': f1,
    }


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
# Training starts here
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.951700,0.908166,0.610108,0.552366
2,0.863200,0.787904,0.669675,0.600319
3,0.697600,0.672428,0.723827,0.699031


TrainOutput(global_step=831, training_loss=0.8375067865805506, metrics={'train_runtime': 12486.94, 'train_samples_per_second': 0.531, 'train_steps_per_second': 0.067, 'total_flos': 436505160010752.0, 'train_loss': 0.8375067865805506, 'epoch': 3.0})

In [ ]:
# Example Hinglish text for prediction
example_text = "iphone 14"
inputs = tokenizer(example_text, return_tensors='pt', padding=True, truncation=True, max_length=128)
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
label_map_reverse = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_label = label_map_reverse[predictions.item()]
print(f"Predicted sentiment: {predicted_label}")


Predicted sentiment: neutral


# **Model v2.0**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/red_data_3.csv')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
train_dataset = dataset.train_test_split(test_size=0.2)['train']
val_dataset = dataset.train_test_split(test_size=0.2)['test']


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_function(examples):
    tokenized = tokenizer(examples['body'], padding='max_length', truncation=True, max_length=128)
    tokenized['label'] = examples['score']
    return tokenized

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/957 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns(['body', 'score'])
val_dataset = val_dataset.remove_columns(['body', 'score'])


In [ ]:
# Set the dataset format to PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [ ]:
print(train_dataset[0])


{'input_ids': tensor([  101, 43958,   169, 12331, 14985, 13262,   119,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
# Set up training arguments with logging and evaluation at every epoch
training_args = TrainingArguments(
    output_dir='./results',                # Output directory for saving model checkpoints
    num_train_epochs=3,                    # Number of training epochs
    per_device_train_batch_size=8,         # Batch size for training
    per_device_eval_batch_size=16,         # Batch size for evaluation
    evaluation_strategy='epoch',           # Evaluate the model at the end of each epoch
    logging_dir='./logs',                  # Directory to store logs
    logging_strategy='epoch',              # Log status after every epoch
    save_strategy='epoch',                 # Save model after every epoch
    load_best_model_at_end=True,           # Load the best model when finished
    metric_for_best_model='accuracy',      # Use accuracy as the evaluation metric
    logging_steps=10,                      # Logging steps (adjust if you want logging during training as well)
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
# Training starts here
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,1.085600,1.073503,0.391667,0.314148
2,0.953700,0.803112,0.554167,0.476434
3,0.733500,0.582669,0.745833,0.735747


TrainOutput(global_step=360, training_loss=0.9242706722683377, metrics={'train_runtime': 4700.9215, 'train_samples_per_second': 0.611, 'train_steps_per_second': 0.077, 'total_flos': 188849655574272.0, 'train_loss': 0.9242706722683377, 'epoch': 3.0})

In [ ]:
# Example Hinglish text for prediction
example_text = "ai"
inputs = tokenizer(example_text, return_tensors='pt', padding=True, truncation=True, max_length=128)
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
label_map_reverse = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_label = label_map_reverse[predictions.item()]
print(f"Predicted sentiment: {predicted_label}")


Predicted sentiment: neutral


# **Model v3.0**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/red_data_3.csv')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

Using device: cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# Convert data to Dataset format compatible with Hugging Face Trainer
def preprocess_data(df):
    return {'text': df['body'], 'label': df['score']}

In [ ]:
train_size = int(0.8 * len(df))
train_data = df[:train_size]
test_data = df[train_size:]

# Apply preprocessing and convert the data into the proper format
preprocessed_train = df[:train_size].apply(preprocess_data, axis=1).to_list()
preprocessed_test = df[train_size:].apply(preprocess_data, axis=1).to_list()
# Convert preprocessed data into Dataset objects
train_dataset = Dataset.from_list(preprocessed_train)
test_dataset = Dataset.from_list(preprocessed_test)

In [ ]:
# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
# Set format to PyTorch tensors for Trainer
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/957 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',               # output directory
    evaluation_strategy="epoch",          # evaluation strategy (evaluate at the end of each epoch)
    learning_rate=2e-5,                   # learning rate
    per_device_train_batch_size=16,       # batch size for training
    per_device_eval_batch_size=16,        # batch size for evaluation
    num_train_epochs=3,                   # number of training epochs
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='./logs',                 # directory for storing logs
    logging_steps=10,                     # log every 10 steps
    save_strategy="epoch"                 # save model at the end of every epoch
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(pred: EvalPrediction):
    predictions, labels = pred.predictions, pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {
        'accuracy': acc,
        'f1_score': f1,
    }


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

In [ ]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.937300,0.971539,0.537500,0.512048
2,0.768800,0.917665,0.575000,0.564106
3,0.532600,0.885031,0.612500,0.611606


TrainOutput(global_step=180, training_loss=0.791616251733568, metrics={'train_runtime': 345.9039, 'train_samples_per_second': 8.3, 'train_steps_per_second': 0.52, 'total_flos': 755398622297088.0, 'train_loss': 0.791616251733568, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_bert_model')
tokenizer.save_pretrained('./fine_tuned_bert_tokenizer')

('./fine_tuned_bert_tokenizer/tokenizer_config.json',
 './fine_tuned_bert_tokenizer/special_tokens_map.json',
 './fine_tuned_bert_tokenizer/vocab.txt',
 './fine_tuned_bert_tokenizer/added_tokens.json')

In [ ]:
# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained('./fine_tuned_bert_model')
tokenizer = BertTokenizer.from_pretrained('./fine_tuned_bert_tokenizer')

In [ ]:
import torch.nn.functional as F

# Function to predict sentiment with percentage outputs
def predict_sentiment_with_percentages(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits     # Get logits from model output
    probs = F.softmax(logits, dim=-1).flatten()     # Convert logits to probabilities using softmax
    sentiment_labels = ['negative', 'neutral', 'positive']     # Map the probabilities to their respective sentiment labels
    sentiment_percentages = {label: round(prob.item() * 100, 2) for label, prob in zip(sentiment_labels, probs)}
    return sentiment_percentages

**Process Starts Here...**

In [ ]:
# Example prediction
text = "Should i buy a tesla as Emmanuel said it is good but it also very costly"
result = predict_sentiment_with_percentages(text)
print(result)

{'negative': 10.2, 'neutral': 9.84, 'positive': 79.97}


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/red_data_4.csv')

sentiment_results = []

# Iterate over the texts in the DataFrame
for index, row in df.iterrows():
    text = row['comment']
    sentiment_percentages = predict_sentiment_with_percentages(text)
    sentiment_results.append({
        'comment': text,
        'negative': sentiment_percentages['negative'],
        'neutral': sentiment_percentages['neutral'],
        'positive': sentiment_percentages['positive']
    })

sentiment_df = pd.DataFrame(sentiment_results)
print(sentiment_df)

                                               comment  negative  neutral  \
0    Dude : "Now give me back my gun"\nRobot : "giv...     28.68    11.26   
1    The last thing you wanna hear an animatronic s...     27.74    15.85   
2    Its all fun and games until the robot doesn't ...     19.25     8.14   
3                     A robot driving a robot is crazy     64.09     8.58   
4               “Now give me back my gun”\nRobot: “No”      9.50    68.02   
..                                                 ...       ...      ...   
995  He is so polite. He asked everyone to please b...     11.83     6.21   
996  Even though somebody else is controlling it fr...     10.89     7.35   
997    2:32 "When you come to market I'll take two" 👴🏻      9.12    74.32   
998                    We got irobots now before GTA 6      6.20    72.67   
999               remote controlled, operator speaking      7.36    62.33   

     positive  
0       60.06  
1       56.41  
2       72.61  
3       27.

In [ ]:
sentiment_df

,comment,negative,neutral,positive
0,"Dude : ""Now give me back my gun""\nRobot : ""giv...",28.68,11.26,60.06
1,The last thing you wanna hear an animatronic s...,27.74,15.85,56.41
2,Its all fun and games until the robot doesn't ...,19.25,8.14,72.61
3,A robot driving a robot is crazy,64.09,8.58,27.33
4,“Now give me back my gun”\nRobot: “No”,9.50,68.02,22.48
...,...,...,...,...
995,He is so polite. He asked everyone to please b...,11.83,6.21,81.96
996,Even though somebody else is controlling it fr...,10.89,7.35,81.76
997,"2:32 ""When you come to market I'll take two"" 👴🏻",9.12,74.32,16.56
998,We got irobots now before GTA 6,6.20,72.67,21.13


In [ ]:
print("Positive: ",sentiment_df['positive'].mean())
print("Neutral: ",sentiment_df['neutral'].mean())
print("Negative: ",sentiment_df['negative'].mean())

Positive:  49.87553
Neutral:  28.14389
Negative:  21.980660000000004
